In [8]:
"""
設定ファイル - Qwen2.5-32B モデルのトレーニングと推論用設定
"""

# Model configuration
VER = "4bit"
MODEL_NAME = "/kaggle/input/qwen2.5/transformers/32b-instruct/1"
MODEL_TYPE = "qwen2"  # Qwen2.5 uses qwen2 architecture
EPOCHS = 3  # Reduce epochs for initial testing
MAX_LEN = 300  # Increase max length for better context

# Directory settings
OUTPUT_DIR = f"/kaggle/input/qwen2.5-32b-0.9485/transformers/fulltrain/1/09485ft"

# Training parameters
TRAIN_BATCH_SIZE = 8  # Batch size 2 for RTX 5090 with 31GB VRAM
EVAL_BATCH_SIZE = 4  # Eval can use larger batch size
GRADIENT_ACCUMULATION_STEPS = 8  # Reduced to 32 for faster training
LEARNING_RATE = 2e-4
LOGGING_STEPS = 50
SAVE_STEPS = 229
EVAL_STEPS = 229


# Data paths
TRAIN_DATA_PATH = '/kaggle/input/map-charting-student-math-misunderstandings/train.csv'
TEST_DATA_PATH = '/kaggle/input/map-charting-student-math-misunderstandings/test.csv'

# Model save paths
BEST_MODEL_PATH = f"{OUTPUT_DIR}/checkpoint-1722"
LABEL_ENCODER_PATH = f"{OUTPUT_DIR}/label_encoder.joblib"

# Other settings
RANDOM_SEED = 42
VALIDATION_SPLIT = 0.2

# GPU settings
CUDA_VISIBLE_DEVICES = "0,1"  # GPU device to use. Set to None to use all available GPUs

# Submission settings
SUBMISSION_OUTPUT_PATH = 'submission.csv'

# WandB settings
USE_WANDB = True  # Set to False to disable WandB
WANDB_PROJECT = "qwen2.5-32b-math-misconceptions"
WANDB_RUN_NAME = f"qwen2.5-32b-ver{VER}"
WANDB_ENTITY = None  # Set your WandB entity (username or team name) if needed

# Early stopping settings
USE_EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 10  # 改善が見られない評価回数の上限（評価はEVAL_STEPSごとに実行される）
EARLY_STOPPING_THRESHOLD = 0.001  # 改善とみなす最小変化量

# Evaluation/Save strategy
EVALUATION_STRATEGY = "steps"
SAVE_STRATEGY = "steps"

# LoRA configuration
LORA_RANK = 128  # LoRAのランク - reduced for memory efficiency
LORA_ALPHA = 128  # LoRAのスケーリングパラメータ - reduced proportionally
LORA_TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]  # 対象モジュール
LORA_DROPOUT = 0.1  # LoRAのドロップアウト率
LORA_BIAS = "none"  # biasの扱い: "none", "all", "lora_only"

# Memory optimization settings
USE_GRADIENT_CHECKPOINTING = True  # Enable gradient checkpointing
USE_8BIT_ADAM = False  # Use 8-bit Adam optimizer for memory efficiency
MAX_GRAD_NORM = 1.0  # Gradient clipping value

# 4-bit quantization settings
# 4-bit quantization settings
USE_4BIT_QUANTIZATION = True  # Enable 4-bit quantization
# GPUとbf16対応状況に応じて自動選択

BNB_4BIT_COMPUTE_DTYPE = "float16"
BNB_4BIT_QUANT_TYPE = "nf4"  # Quantization type (fp4 or nf4)
BNB_4BIT_USE_DOUBLE_QUANT = True  # Use double quantization
BNB_4BIT_QUANT_STORAGE_DTYPE = "uint8"  # Storage dtype for quantized weights


In [9]:
!pip install --no-index --no-deps /kaggle/input/bitsandbytes-20250725/bitsandbytes/bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl

Processing /kaggle/input/bitsandbytes-20250725/bitsandbytes/bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl
bitsandbytes is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [10]:
"""
共通ユーティリティ関数
"""

import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from datasets import Dataset
import torch


def prepare_correct_answers(train_data):
    """正解答案データを準備"""
    idx = train_data.apply(lambda row: row.Category.split('_')[0] == 'True', axis=1)
    correct = train_data.loc[idx].copy()
    correct['c'] = correct.groupby(['QuestionId','MC_Answer']).MC_Answer.transform('count')
    correct = correct.sort_values('c', ascending=False)
    correct = correct.drop_duplicates(['QuestionId'])[['QuestionId','MC_Answer']]
    correct['is_correct'] = 1
    return correct


def format_input(row):
    """入力データをモデル用プロンプトにフォーマット"""
    if row["is_correct"]:
        status = "Yes"
    else:
        status = "No"

    # Qwen2.5用の数学誤解分析タスクに特化したプロンプト
    prompt = (
        "<|im_start|>user"
        f"[Mathematical Misconception Analysis Task]\n\n"
        f"Question: {row['QuestionText']}\n"
        f"Answer: {row['MC_Answer']}\n"
        f"Correct?: {status}\n"
        f"Explanation: {row['StudentExplanation']}\n\n"
        "<|im_end|>\n<|im_start|>assistant\n"
        "<think>\n"
        "Let me analyze this mathematical misconception...\n"
        "</think>\n\n"
        "<|im_end|>"
    )
    return prompt


def tokenize_dataset(dataset, tokenizer, max_len):
    """データセットをトークナイズ"""
    def tokenize(batch):
        # パディングはDataCollatorで行うため、ここではトークナイズのみ
        return tokenizer(
            batch['text'],
            padding=False,  # パディングはDataCollatorに任せる
            truncation=True,
            max_length=max_len,
            return_tensors=None  # map時は'None'を使用
        )

    dataset = dataset.map(tokenize, batched=True, batch_size=100)
    # columnsの設定時にlabelを保持
    columns = ['input_ids', 'attention_mask', 'label'] if 'label' in dataset.column_names else ['input_ids', 'attention_mask']
    dataset.set_format(type='torch', columns=columns)
    return dataset


def compute_map3(eval_pred):
    """Top-3 予測に基づくMAP@3を計算"""
    logits, labels = eval_pred
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    top3 = np.argsort(-probs, axis=1)[:, :3]
    score = 0.0
    for i, label in enumerate(labels):
        ranks = top3[i]
        if ranks[0] == label:
            score += 1.0
        elif ranks[1] == label:
            score += 1.0 / 2
        elif ranks[2] == label:
            score += 1.0 / 3
    return {"map@3": score / len(labels)}


def create_submission(predictions, test_data, label_encoder):
    """予測結果から提出用ファイルを作成"""
    probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()
    top3 = np.argsort(-probs, axis=1)[:, :3]
    flat = top3.flatten()
    decoded = label_encoder.inverse_transform(flat)
    top3_labels = decoded.reshape(top3.shape)
    pred_strings = [" ".join(r) for r in top3_labels]

    submission = pd.DataFrame({
        'row_id': test_data.row_id.values,
        'Category:Misconception': pred_strings
    })
    return submission


In [11]:
"""
Qwen2.5-32B モデル推論スクリプト - 提出用予測ファイルの生成
"""

import pandas as pd
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
import time
import joblib
import torch
import gc
# PEFT は必須。未インストール時は ImportError がそのまま表示されます。
from peft import PeftModel, PeftConfig


# Kaggle環境ではカスタムクラスは不要


def main():
    """メイン推論関数"""

    # メモリキャッシュをクリア
    torch.cuda.empty_cache()
    gc.collect()

    # CUDAメモリ管理の最適化
    import os
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    # 2つのGPUを使用可能にする
    if torch.cuda.device_count() > 1:
        print(f"Found {torch.cuda.device_count()} GPUs")

    print("Loading label encoder...")
    # ラベルエンコーダーの読み込み
    le = joblib.load(LABEL_ENCODER_PATH)
    n_classes = len(le.classes_)

    print("Loading trained model and tokenizer...")

    # LoRAアダプターを使用する場合（PEFT は必須）
    print(f"Loading fine-tuned LoRA model from: {BEST_MODEL_PATH}")
    print(f"Loading base model from: {MODEL_NAME}")

    # ベースモデルを読み込む（4bit量子化で読み込み）
    from transformers import BitsAndBytesConfig

    # config.pyの設定に基づいて4bit量子化を構成
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=getattr(torch, BNB_4BIT_COMPUTE_DTYPE),
        bnb_4bit_quant_type=BNB_4BIT_QUANT_TYPE,
        bnb_4bit_use_double_quant=BNB_4BIT_USE_DOUBLE_QUANT,
        bnb_4bit_quant_storage_dtype=getattr(torch, BNB_4BIT_QUANT_STORAGE_DTYPE)
    )

    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=n_classes,
            trust_remote_code=True,
            quantization_config=quantization_config,
            device_map="auto",  # 自動的に複数GPUに分散
            low_cpu_mem_usage=True  # CPUメモリ使用量を削減
        )
    except ValueError as e:
        # 4bit以外へのフォールバックは行わない方針。
        # Kaggle等で "dispatched on the CPU or the disk" が発生する場合は
        # VRAMやバッチ/シーケンス長の見直しを促し、明示的に失敗させる。
        if "dispatched on the CPU or the disk" in str(e):
            raise RuntimeError(
                "4bitロードに失敗しました。8bitフォールバックは無効です。"
                "VRAMを増やすか、MAX_LEN/EVAL_BATCH_SIZEなどを下げて再実行してください。"
            ) from e
        else:
            raise

    # LoRAアダプターを適用
    model = PeftModel.from_pretrained(model, BEST_MODEL_PATH)
    # 注意: 4bit量子化モデル + LoRA を Trainer に渡す際、
    # LoRA をベースへマージすると「純量子化モデル」扱いになり Trainer 初期化で失敗する。
    # そのため推論では必ず PEFT ラッパーを保持し、マージしない。

    # 推論モードに設定（メモリ効率化）
    model.eval()
    # 4bit量子化モデルは既にGPUに配置されているのでto('cuda')は不要

    # トークナイザーはベースモデルから読み込む
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print("Successfully loaded LoRA fine-tuned model")

    # パディングトークンの設定
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # モデルの設定を更新（PeftModelのbase_modelにアクセス）
    if hasattr(model, 'base_model'):
        model.base_model.config.pad_token_id = tokenizer.pad_token_id
        # 内部のモデルにも設定
        if hasattr(model.base_model, 'model'):
            model.base_model.model.config.pad_token_id = tokenizer.pad_token_id
    else:
        model.config.pad_token_id = tokenizer.pad_token_id

    print("Loading test data...")
    # テストデータの読み込み
    test = pd.read_csv(TEST_DATA_PATH)

    print("Loading training data for correct answers...")
    # 正解答案データの準備（訓練データから取得）
    train = pd.read_csv(TRAIN_DATA_PATH)
    train.Misconception = train.Misconception.fillna('NA')
    correct = prepare_correct_answers(train)

    print("Preprocessing test data...")
    # テストデータの前処理
    test = test.merge(correct, on=['QuestionId','MC_Answer'], how='left')
    test.is_correct = test.is_correct.fillna(0)
    test['text'] = test.apply(format_input, axis=1)

    print("Tokenizing test data...")
    # テストデータのトークナイズ
    ds_test = Dataset.from_pandas(test[['text']])
    ds_test = tokenize_dataset(ds_test, tokenizer, MAX_LEN)

    # パディングのためのデータコラレータの設定
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    print("Running inference...")

    # TF32を有効化（推論速度向上）
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # 推論の実行
    trainer = Trainer(
        model=model,
        processing_class=tokenizer,  # tokenizer の代替
        data_collator=data_collator,  # バッチ時に自動でパディングを適用
        args=TrainingArguments(
            output_dir="./tmp",  # 一時ディレクトリ（必須パラメータ）
            report_to="none",    # wandbを無効化
            per_device_eval_batch_size=EVAL_BATCH_SIZE,  # 設定ファイルから取得
            fp16=True,  # float16を使用
            dataloader_pin_memory=True,  # データローダーの高速化
            dataloader_num_workers=2,  # データ読み込みの並列化
        )
    )
    # inference_mode で推論を実行（さらに軽量な no_grad）
    with torch.inference_mode():
        # モデルロード後の1件あたり推論時間を計測（データ全体を一括推論した実効平均）
        start = time.perf_counter()
        predictions = trainer.predict(ds_test)
        try:
            if torch.cuda.is_available():
                torch.cuda.synchronize()
        except Exception:
            pass
        elapsed = time.perf_counter() - start
        num_samples = len(ds_test)
        if num_samples > 0:
            ms_per_sample = (elapsed / num_samples) * 1000.0
            print(f"Average inference time: {ms_per_sample:.2f} ms/sample (batch={EVAL_BATCH_SIZE}, samples={num_samples})")

    print("Creating submission file...")
    # 提出用ファイルの作成
    submission = create_submission(predictions, test, le)

    # ファイルの保存
    submission.to_csv(SUBMISSION_OUTPUT_PATH, index=False)
    print(f"Submission file saved to: {SUBMISSION_OUTPUT_PATH}")
    print("\nSubmission preview:")
    print(submission.head())
    print(f"\nSubmission shape: {submission.shape}")


if __name__ == "__main__":
    main()


Found 2 GPUs
Loading label encoder...
Loading trained model and tokenizer...
Loading fine-tuned LoRA model from: /kaggle/input/qwen2.5-32b-0.9485/transformers/fulltrain/1/09485ft/checkpoint-1722
Loading base model from: /kaggle/input/qwen2.5/transformers/32b-instruct/1


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`